In [172]:
import os 
from dotenv import load_dotenv
import requests
from google.oauth2 import service_account
import xmltodict
import json
import pandas as pd 
from requests import request
import pytz
from bs4 import BeautifulSoup
from typing import Dict
from google.oauth2 import service_account
from google.cloud import storage

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import sys

from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType

from entsoe import EntsoeRawClient
from entsoe import EntsoePandasClient





In [173]:
#load env variables
load_dotenv('./creds/.env', verbose=True, override=True)

True

In [3]:
print(os.environ.get("PYSPARK_DRIVER_PYTHON"))

None


In [174]:


'''
----------------
INIT VARIABLES
----------------
'''

#setting up entsoe variables
security_token = os.environ.get("SECURITY_TOKEN")
ENTSOE_URL = 'https://transparency.entsoe.eu/api'

#setting up GCP variables
service_account_file = os.environ.get("SERVICE_ACCOUNT_FILE")
credentials = service_account.Credentials.from_service_account_file(
    service_account_file
)
gcs_bucket = os.environ.get("CLOUD_STORAGE_BUCKET")
print(gcs_bucket)

entsoe_analytics_1009


In [ ]:



# #setting up session
# entsoe_session = requests.Session()

# '''
# ----------------
# SETTING UP FUNCTION CALLS 
# ----------------
# '''

# #format dates
# def datetime_to_str(dtm: pd.Timestamp) -> str:
#     #convert timezone to UTC if it's exist and it's not on UTC already
#     if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
#         dtm = dtm.tz_convert("UTC")
#     fmt = '%Y%m%d%H%M'
#     dtm_str = dtm.strftime(fmt)
#     return dtm_str

# #basic requests
# def basic_requests(params: Dict, start:pd.Timestamp, end: pd.Timestamp) -> requests.Response:
#     #setting up time intervals start and stop
#     start_str = datetime_to_str(start)
#     end_str = datetime_to_str(end)

#     #setting up params and extending with custom parameter based
#     base_params = {
#         'securityToken': security_token,
#         'periodStart': start_str,
#         'periodEnd': end_str,
#     }
#     params.update(base_params)
    
#     #seting up sesssion
#     session = requests.Session()
#     response = session.get(url=ENTSOE_URL, params=params)

#     return response

# # upload data to GCS
# def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
#     # Upload file to bucket"""

#     # ID of GCS bucket
#     # bucket_name =

#     # the contents from memory to be uploaded to file
#     # contents =

#     # the ID of your GCS object
#     # destination_blob_name =

#     storage_client = storage.Client(credentials=credentials)
#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(destination_blob_name)

#     blob.upload_from_string(contents)


# '''
# ----------------
# EXTRACTION
# ----------------
# '''
# #for test, we'll be querying Actual Generation Output per Generation Unit

# domain = '10YCZ-CEPS-----N'
# params_requests = {
#     'documentType': 'A73',
#     'processType': 'A16',
#     'in_Domain': {domain},
# }
# start=pd.Timestamp('202101011200', tz='Europe/Berlin')
# end=pd.Timestamp('202101011300', tz='Europe/Berlin')

# entsoe_data = basic_requests(params=params_requests, start=start, end=end)
# # entsoe_dict = xmltodict.parse(entsoe_data.text)
# # entsoe_json = json.dumps(entsoe_dict, indent=4)
# # print(entsoe_json)
# print(entsoe_data.text)

# '''
# ----------------
# LOAD
# ----------------
# '''
# # #upload to GCS
# # landing_filename=f"entsoe_data_{start}.json"
# # upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

# #upload to GCS - XML
# landing_filename=f"entsoe_data_{start}_{domain}.xml"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data.text, destination_blob_name=landing_filename)



In [188]:



#setting up session
entsoe_client = EntsoePandasClient(security_token)
'''
----------------
SETTING UP FUNCTION CALLS 
----------------
'''

# upload data to GCS
def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
    # Upload file to bucket"""

    # ID of GCS bucket
    # bucket_name =

    # the contents from memory to be uploaded to file
    # contents =

    # the ID of your GCS object
    # destination_blob_name =

    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)


'''
----------------
EXTRACTION
----------------
'''
#for test, we'll be querying Actual Generation Output per Generation Unit

start=pd.Timestamp('202101011200', tz='Europe/Berlin')
end=pd.Timestamp('202101011300', tz='Europe/Berlin')
country_code= 'CZ'
country_code_from=''
country_code_to=''
type_marketagreement_type=''
contract_marketagreement_type=''

try:
    entsoe_data = entsoe_client.query_generation_per_plant(country_code, start=start, end=end)
    display(entsoe_data)
    entsoe_json = entsoe_data.to_json()
except Exception as e:
    print("An exception occurred:", e)


# '''
# ----------------
# LOAD
# ----------------
# '''
# #upload to GCS
# landing_filename=f"entsoe_data_{country_code}.json"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

# # #upload to GCS - XML
# # landing_filename=f"entsoe_data_{start}_{country_code}.xml"
# # upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data, destination_blob_name=landing_filename)



,ECHV_G1____,ECHV_G2____,ECHV_G3____,ECHV_G4____,EDAL_G1____,EDAL_G2____,EDAL_G3____,EDAL_G4____,EDET_G2____,EDET_G3____,...,EPR2_G25___,EPVR.B1,EPVR.B2,ETEM_G1____,ETEM_G2____,ETI2.G6,ETU2_G21___,ETU2_G22___,ETU2_G23___,ETU2_G24___
,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite,Hydro Pumped Storage,Hydro Pumped Storage,Hydro Pumped Storage,Hydro Pumped Storage,Fossil Hard coal,Fossil Hard coal,...,Fossil Brown coal/Lignite,Fossil Gas,Fossil Gas,Nuclear,Nuclear,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite,Fossil Brown coal/Lignite
newlevel,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,...,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated
2021-01-01 12:00:00+01:00,155.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,215.0,124.0,124.0,1035.0,1047.0,69.0,167.0,167.0,176.0,176.0


In [6]:
print(os.environ["SPARK_HOME"])
SPARK_HOME = os.environ["SPARK_HOME"]

/opt/spark-3.3.1-bin-hadoop3-scala2.13


In [7]:

# #setup sparksession for entry point
# SPARK_HOME = os.environ["SPARK_HOME"]
# spark = SparkSession.builder.appName("gcp_playground").config("spark.jars", f"{SPARK_HOME}/jars/spark-xml_2.13-0.15.0.jar").getOrCreate()



In [7]:
## COBA SPARK GCS CONNECTOR ##


# berangkat pak haji
# setup parameternya
gcs_bucket = gcs_bucket
path = f"gs://{gcs_bucket}/{landing_filename}"
# path ="/home/rafzul/projects/entsoe-pipelines/sample.xml"

#coba spark gcs connector
#setup sparksession for entry point - COBA GCS CONNECTOR
SPARK_HOME = os.environ["SPARK_HOME"]
spark = SparkSession.builder.appName("gcp_playground") \
    .config("spark.jars", f"{SPARK_HOME}/jars/gcs-connector-hadoop3-latest.jar") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", service_account_file) \
    .getOrCreate()



22/12/27 20:03:53 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.18.2 instead (on interface wlp58s0)
22/12/27 20:03:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/27 20:03:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
print(spark.conf.get("spark.hadoop.fs.AbstractFileSystem.gs.impl"))

google.cloud.hadoop.fs.gcs.GoogleHadoopFS


In [166]:
path = f"gs://{gcs_bucket}/{landing_filename}"
print(path)

#create dataframe from gcs
df_spark = spark.read.format("json") \
   .option("header","true") \
   .option("multiLine","true") \
   .option("inferSchema","true") \
   .load(path)
   

gs://entsoe_analytics_1009/entsoe_data_CZ.json


In [167]:
from pyspark.sql import functions as F

In [168]:
df_spark.printSchema()
df_spark.dtypes
df_spark.show()

root
 |-- GL_MarketDocument: struct (nullable = true)
 |    |-- @xmlns: string (nullable = true)
 |    |-- TimeSeries: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- MktPSRType: struct (nullable = true)
 |    |    |    |    |-- PowerSystemResources: struct (nullable = true)
 |    |    |    |    |    |-- mRID: struct (nullable = true)
 |    |    |    |    |    |    |-- #text: string (nullable = true)
 |    |    |    |    |    |    |-- @codingScheme: string (nullable = true)
 |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- psrType: string (nullable = true)
 |    |    |    |-- Period: struct (nullable = true)
 |    |    |    |    |-- Point: struct (nullable = true)
 |    |    |    |    |    |-- position: string (nullable = true)
 |    |    |    |    |    |-- quantity: string (nullable = true)
 |    |    |    |    |-- resolution: string (nullable = true)
 |    |    |    |    |-- timeInterval: struct (nul

In [170]:
df_spark = df_spark.select("GL_MarketDocument.*")


In [171]:
# abc.show(50,truncate=False)
print(df_spark.dtypes)
array_cols = [c[0] for c in df_spark.dtypes if c[1][:5] == 'array'] 
print(array_cols)

for array_col in array_cols:
            
    # cols_to_select = [x for x in df.columns if x != array_col ]
    df_spark = df_spark.withColumn(array_col, F.explode(F.col(array_col)))
    
df_spark.show(50)
df_spark.dtypes

[('@xmlns', 'string'), ('TimeSeries', 'array<struct<MktPSRType:struct<PowerSystemResources:struct<mRID:struct<#text:string,@codingScheme:string>,name:string>,psrType:string>,Period:struct<Point:struct<position:string,quantity:string>,resolution:string,timeInterval:struct<end:string,start:string>>,businessType:string,curveType:string,inBiddingZone_Domain.mRID:struct<#text:string,@codingScheme:string>,mRID:string,objectAggregation:string,quantity_Measure_Unit.name:string,registeredResource.mRID:struct<#text:string,@codingScheme:string>>>'), ('createdDateTime', 'string'), ('mRID', 'string'), ('process.processType', 'string'), ('receiver_MarketParticipant.mRID', 'struct<#text:string,@codingScheme:string>'), ('receiver_MarketParticipant.marketRole.type', 'string'), ('revisionNumber', 'string'), ('sender_MarketParticipant.mRID', 'struct<#text:string,@codingScheme:string>'), ('sender_MarketParticipant.marketRole.type', 'string'), ('time_Period.timeInterval', 'struct<end:string,start:string>')

[('@xmlns', 'string'),
 ('TimeSeries',
  'struct<MktPSRType:struct<PowerSystemResources:struct<mRID:struct<#text:string,@codingScheme:string>,name:string>,psrType:string>,Period:struct<Point:struct<position:string,quantity:string>,resolution:string,timeInterval:struct<end:string,start:string>>,businessType:string,curveType:string,inBiddingZone_Domain.mRID:struct<#text:string,@codingScheme:string>,mRID:string,objectAggregation:string,quantity_Measure_Unit.name:string,registeredResource.mRID:struct<#text:string,@codingScheme:string>>'),
 ('createdDateTime', 'string'),
 ('mRID', 'string'),
 ('process.processType', 'string'),
 ('receiver_MarketParticipant.mRID',
  'struct<#text:string,@codingScheme:string>'),
 ('receiver_MarketParticipant.marketRole.type', 'string'),
 ('revisionNumber', 'string'),
 ('sender_MarketParticipant.mRID',
  'struct<#text:string,@codingScheme:string>'),
 ('sender_MarketParticipant.marketRole.type', 'string'),
 ('time_Period.timeInterval', 'struct<end:string,start:

In [128]:
abc = df_spark.select("GL_MarketDocument.*").select("@xmlns", F.explode("TimeSeries").alias("Nilai"))

In [105]:
print(abc.dtypes)
abc.show(50)
abc.printSchema()

[('@xmlns', 'string'), ('Nilai', 'struct<MktPSRType:struct<PowerSystemResources:struct<mRID:struct<#text:string,@codingScheme:string>,name:string>,psrType:string>,Period:struct<Point:struct<position:string,quantity:string>,resolution:string,timeInterval:struct<end:string,start:string>>,businessType:string,curveType:string,inBiddingZone_Domain.mRID:struct<#text:string,@codingScheme:string>,mRID:string,objectAggregation:string,quantity_Measure_Unit.name:string,registeredResource.mRID:struct<#text:string,@codingScheme:string>>')]
+--------------------+--------------------+
|              @xmlns|               Nilai|
+--------------------+--------------------+
|urn:iec62325.351:...|{{{{27W-GU-ECHVG1...|
|urn:iec62325.351:...|{{{{27W-GU-ECHVG2...|
|urn:iec62325.351:...|{{{{27W-GU-ECHVG3...|
|urn:iec62325.351:...|{{{{27W-GU-ECHVG4...|
|urn:iec62325.351:...|{{{{27W-GU-EDALG1...|
|urn:iec62325.351:...|{{{{27W-GU-EDALG2...|
|urn:iec62325.351:...|{{{{27W-GU-EDALG3...|
|urn:iec62325.351:...|{{{{2

In [ ]:
#inferring schema and get the data type of each column and turn it into spark dataframe
datatype_infer = pd.DataFrame.from_dict(xml_file_dict[0], orient='index')

In [ ]:
 df_spark.schema.prettyJson()

In [ ]:
#flatten nested df at every layer
from pyspark.sql.types import *
from pyspark.sql.import functions as f

def flatten_structs(nested_df):
    stack = [(), nested_df]
    columns = []
    
    while len(stack) > 0:
        parents
        
    